# Point cloud segmentation

## Imports

In [2]:
import open3d as o3d
import sys
import numpy as np
from sklearn.linear_model import RANSACRegressor
import random

## Loading in the data

In [3]:
# Load in point cloud
pcd = o3d.io.read_point_cloud("../data/cube1.ply")
pcd.paint_uniform_color([0.9, 0.9, 0.9])
points = np.asarray(pcd.points)

## RANSAC
### SciKit Learn

In [4]:
# Fit a plane to the point cloud
ransac = RANSACRegressor(min_samples=3, residual_threshold=0.01, max_trials=1000)
ransac.fit(points[:, :2], points[:, 2])

# Get the inlier points
inlier_mask = ransac.inlier_mask_
inlier_points = points[inlier_mask, :]

# Get the outlier points
outlier_mask = np.logical_not(ransac.inlier_mask_)
outlier_points = points[outlier_mask, :]

# Get the plane parameters
plane = ransac.estimator_.coef_
plane = np.append(plane, ransac.estimator_.intercept_)

In [5]:
# Create a new point cloud with only the inlier points
inlier_pcd = o3d.geometry.PointCloud()
inlier_pcd.points = o3d.utility.Vector3dVector(inlier_points)
inlier_pcd.paint_uniform_color([0.9, 0, 0])

# Create a new point cloud with only the outlier points
outlier_pcd = o3d.geometry.PointCloud()
outlier_pcd.points = o3d.utility.Vector3dVector(outlier_points)
outlier_pcd.paint_uniform_color([0, 0.9, 0])

# Visualize the point cloud
o3d.visualization.draw_geometries([outlier_pcd, inlier_pcd])

### Open3D

In [6]:
# Use RANSAC to segment the plane
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)

# Extract the inlier points
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

# Extract the outlier points
outlier_cloud = pcd.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0, 0, 1])

# Visualize the point cloud
o3d.visualization.draw_geometries([outlier_cloud, inlier_cloud])

We can see that the open3D implementation, instantly detects a plane whereas this is not the case with sci-kit learn. However tweaking the parameters of the sci-kit learn implementation can lead to a similar result. Using the sci-kit learn package may lead to better results in the future as it is more flexible and can be used for other algorithms.

## Finding all planes

Now we are going to loop the algorithm over the entire point cloud and find all planes. We will try both the sci-kit learn and open3D implementations.

### SciKit Learn

In [7]:
# Load in point cloud
pcd = o3d.io.read_point_cloud("../data/cube1.ply")
pcd.estimate_normals()

planes = []

# Fit a plane to the point cloud
ransac = RANSACRegressor(min_samples=3, residual_threshold=0.01, max_trials=1000)
ransac.fit(points[:, :2], points[:, 2])

# Get the inlier points
inlier_mask = ransac.inlier_mask_
inlier_points = points[inlier_mask, :]

# Get the outlier points
outlier_mask = np.logical_not(ransac.inlier_mask_)
outlier_points = points[outlier_mask, :]

# Create a new point cloud with only the inlier points
inlier_pcd = o3d.geometry.PointCloud()
inlier_pcd.points = o3d.utility.Vector3dVector(inlier_points)
inlier_pcd.paint_uniform_color([0.9, 0, 0])

# Add the plane to the list of planes
planes.append(inlier_pcd)

# Repeat until there are no more points
# while len(outlier_points) > 0:
# Fit a plane to the point cloud
ransac = RANSACRegressor(min_samples=3, residual_threshold=0.01, max_trials=1000)
ransac.fit(outlier_points[:, :2], outlier_points[:, 2])

# Get the inlier points
inlier_mask = ransac.inlier_mask_
inlier_points = outlier_points[inlier_mask, :]

# Get the outlier points
outlier_mask = np.logical_not(ransac.inlier_mask_)
outlier_points = outlier_points[outlier_mask, :]

# Create a new point cloud with only the inlier points
inlier_pcd = o3d.geometry.PointCloud()
inlier_pcd.points = o3d.utility.Vector3dVector(inlier_points)
inlier_pcd.paint_uniform_color([0, 0.9, 0])

# Add the plane to the list of planes
planes.append(inlier_pcd)

ransac = RANSACRegressor(min_samples=3, residual_threshold=0.025, max_trials=1000)
ransac.fit(outlier_points[:, :2], outlier_points[:, 2])

# Get the inlier points
inlier_mask = ransac.inlier_mask_
inlier_points = outlier_points[inlier_mask, :]

# Get the outlier points
outlier_mask = np.logical_not(ransac.inlier_mask_)
outlier_points = outlier_points[outlier_mask, :]

# Create a new point cloud with only the inlier points
inlier_pcd = o3d.geometry.PointCloud()
inlier_pcd.points = o3d.utility.Vector3dVector(inlier_points)
inlier_pcd.paint_uniform_color([0, 0, 0.9])

# Add the plane to the list of planes
planes.append(inlier_pcd)

# Visualize the point cloud
o3d.visualization.draw_geometries(planes)

### Open3D



In [8]:
# Load in point cloud
pcd = o3d.io.read_point_cloud("../data/cube1.ply")
planes = []

while len(pcd.points) > 0:
    # Use RANSAC to segment the plane
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)

    # Extract the inlier points
    inlier_cloud = pcd.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([random.random(), random.random(), random.random()])

    # Extract the outlier points
    pcd = pcd.select_by_index(inliers, invert=True)

    # Add the plane to the list of planes
    planes.append(inlier_cloud)

print(len(planes))

# Visualize the point cloud
o3d.visualization.draw_geometries(planes)


6


The open3D implementation is much easier to use, after tinkering with the parameters of the sci-kit learn implementation I was still unable to get the same results, therefore I will be using the open3D implementation for the rest of the notebook.

Let's try to use the open3D implementation to find all planes other point clouds.

In [9]:
# Load in point cloud
pcd = o3d.io.read_point_cloud("../data/cube4.ply")
planes = []

while len(pcd.points) > 0:
    # Use RANSAC to segment the plane
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)

    # Extract the inlier points
    inlier_cloud = pcd.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([random.random(), random.random(), random.random()])

    # Extract the outlier points
    pcd = pcd.select_by_index(inliers, invert=True)

    # Add the plane to the list of planes
    planes.append(inlier_cloud)

print(len(planes))

# Visualize the point cloud
o3d.visualization.draw_geometries(planes)

6


In [10]:
# Load in point cloud
pcd = o3d.io.read_point_cloud("../data/pyramid1.ply")
planes = []

while len(pcd.points) > 0:
    # Use RANSAC to segment the plane
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)

    # Extract the inlier points
    inlier_cloud = pcd.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([random.random(), random.random(), random.random()])

    # Extract the outlier points
    pcd = pcd.select_by_index(inliers, invert=True)

    # Add the plane to the list of planes
    planes.append(inlier_cloud)

print(len(planes))

# Visualize the point cloud
o3d.visualization.draw_geometries(planes)


5


In [22]:
# Load in point cloud
pcd = o3d.io.read_point_cloud("../data/noisy_cube2.ply")
planes = []

print(pcd.points)

# Preprocess the point cloud
pcd.voxel_down_sample(voxel_size=0.01)
pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd.remove_radius_outlier(nb_points=16, radius=0.05)


while len(pcd.points) >= 3:
    # Use RANSAC to segment the plane
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)

    # Extract the inlier points
    inlier_cloud = pcd.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([random.random(), random.random(), random.random()])

    # Extract the outlier points
    pcd = pcd.select_by_index(inliers, invert=True)

    # Add the plane to the list of planes
    planes.append(inlier_cloud)

print(len(planes))

# Visualize the point cloud
o3d.visualization.draw_geometries(planes)


std::vector<Eigen::Vector3d> with 6000 elements.
Use numpy.asarray() to access data.
21


Even though there are a lot of planes that seem to be overlapping. All the selected planes do seem to be correct planes. Perhaps if we use a clustering algorithm to find the planes that are overlapping we can get a better result.